In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib3
urllib3.disable_warnings()
import numpy as np
import datetime as dt     
date = dt.date.today
# 60 city agencies

In [2]:
# url="https://a860-openrecords.nyc.gov/request/view_all"
# json_url="https://a860-openrecords.nyc.gov/search/requests?query=&title=on&agency_request_summary=on&open=on&closed=on&date_rec_from=03%2F01%2F2022&date_rec_to=03%2F31%2F2022&agency_ein=&size=50&tz_name=America%2FNew_York&start="
# pagination=requests.get(url)
# page_no=BeautifulSoup(pagination.text)

In [3]:
# creating a dates dataset that can be used to grab monthly data because of 9000 entry limit

d22=pd.DataFrame({"month":np.arange(1,4),"end_date":31, "year":"2022"})
d22['month']=("0"+d22.month.astype(str)).str.replace("010", "10").replace("011", "11").replace("012", "12")
d21=pd.DataFrame({"month":np.arange(1,13),"end_date":31, "year":"2021"})
d21['month']=("0"+d21.month.astype(str)).str.replace("010", "10").replace("011", "11").replace("012", "12")
d20=pd.DataFrame({"month":np.arange(1,13), "end_date":31, "year":"2020"})
d20['month']=("0"+d20.month.astype(str)).str.replace("010", "10").replace("011", "11").replace("012", "12")
d19=pd.DataFrame({"month":np.arange(1,13), "end_date":31, "year":"2019"})
d19['month']=("0"+d19.month.astype(str)).str.replace("010", "10").replace("011", "11").replace("012", "12")
d18=pd.DataFrame({"month":np.arange(1,13),"end_date":31, "year":"2018"})
d18['month']=("0"+d18.month.astype(str)).str.replace("010", "10").replace("011", "11").replace("012", "12")

# 03%2F01%2F2022
d22["url_start_date"]=d22.month+"%2F01%2F"+"2022"
d21["url_start_date"]=d21.month+"%2F01%2F"+"2021"
d20["url_start_date"]=d20.month+"%2F01%2F"+"2020"
d19["url_start_date"]=d19.month+"%2F01%2F"+"2019"
d18["url_start_date"]=d18.month+"%2F01%2F"+"2018"

# 03%2F01%2F2022
d22["url_end_date"]=(d22.month+"%2F31%2F"+"2022").str.replace("02%2F31%2F","02%2F28%2F")
d21["url_end_date"]=(d21.month+"%2F31%2F"+"2021").str.replace("02%2F31%2F","02%2F28%2F")
d20["url_end_date"]=(d20.month+"%2F31%2F"+"2020").str.replace("02%2F31%2F","02%2F28%2F")
d19["url_end_date"]=(d19.month+"%2F31%2F"+"2019").str.replace("02%2F31%2F","02%2F28%2F")
d18["url_end_date"]=(d18.month+"%2F31%2F"+"2018").str.replace("02%2F31%2F","02%2F28%2F")
d22['url_end_date']=d22.url_end_date.str.replace("02%2F31%2F","02%2F28%2F")

merged=pd.concat([d22, d21, d20, d19, d18])

days30=merged[merged.month.isin(["04", "06","09","11"])]
days31=merged[~merged.month.isin(["04", "06","09","11"])]

days30['url_end_date']=days30.url_end_date.str.replace("%2F31%2F", "%2F30%2F")
final_dates=pd.concat([days30, days31]).reset_index(drop=True)
final_dates=final_dates[["url_start_date", "url_end_date"]]

final_dates['start_date']=pd.to_datetime(final_dates.url_start_date.str.replace("%2F", "-"))
final_dates['end_date']=pd.to_datetime(final_dates.url_end_date.str.replace("%2F", "-"))

final_dates['dates']="date_rec_from="+final_dates.url_start_date+"&date_rec_to="+ final_dates.url_end_date

final_dates=final_dates[[ 'start_date', 'end_date','dates']].sort_values(
    "end_date", ascending=False).reset_index(drop=True)

final_dates.to_csv("data/dates.csv", index=False)

/var/folders/09/0pcf7rgs7c38nk2clfp7p_5w0000gq/T/ipykernel_6702/2311917654.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days30['url_end_date']=days30.url_end_date.str.replace("%2F31%2F", "%2F30%2F")


In [4]:
# final_dates=pd.read_csv("data/dates.csv")

base="https://a860-openrecords.nyc.gov/search/requests?query=&title=on&agency_request_summary=on&open=on&closed=on&"
sum_counter=0
table_list=[]

monthly_list=[]

for link in final_dates.dates: 
    URLs=f"{base}{link}&agency_ein=&size=50&tz_name=America%2FNew_York&start="

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko)\
        Chrome/91.0.4472.77 Safari/537.36',
    }
    response = requests.get(URLs,headers=headers,verify=False)
    

    total_entries=int(response.json()['total'])
    sum_counter=sum_counter+total_entries
    
    print(link, sum_counter)
    
    monthly_dict={}
    monthly_dict['date']=link
    monthly_dict['foil_count']=sum_counter
    
    monthly_list.append(monthly_dict)

    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko)\
        Chrome/91.0.4472.77 Safari/537.36',
    }


    for i in np.arange(0, total_entries, 50):
        response = requests.get(f"{base}{link}&agency_ein=&size=50&tz_name=America%2FNew_York&start={i}",
                                headers=headers,verify=False)
        doc=BeautifulSoup(response.json()['results'])
        for tr in doc.find_all('tr'):
            table_dict={}
            table_dict['status']=tr.find_all('td')[0].text.strip()
            table_dict['foil_id']=tr.find_all('td')[1].text.strip()
            table_dict['date_submitted']=tr.find_all('td')[2].text.strip()
            table_dict['title']=tr.find_all('td')[3].text.strip()
            table_dict['ass_agency']=tr.find_all('td')[4].text.strip()
            table_dict['due_date']=tr.find_all('td')[5].text.strip()
            table_dict['href_id']=tr.find_all('td')[3].find('a')['href'].strip()
            table_list.append(table_dict)
    


df=pd.DataFrame(table_list)

# url pattern: https://a860-openrecords.nyc.gov/request/view/FOIL-2022-057-03974

df['foil_link']='https://a860-openrecords.nyc.gov'+df.href_id
df['date_submitted']=pd.to_datetime(df.date_submitted)
df['due_date']=pd.to_datetime(df.due_date)
df=df.sort_values("date_submitted", ascending=False).reset_index(drop=True)

monthly_count=pd.DataFrame(monthly_list)

date_rec_from=03%2F01%2F2022&date_rec_to=03%2F31%2F2022 5813
date_rec_from=02%2F01%2F2022&date_rec_to=02%2F28%2F2022 11494
date_rec_from=01%2F01%2F2022&date_rec_to=01%2F31%2F2022 16773
date_rec_from=12%2F01%2F2021&date_rec_to=12%2F31%2F2021 22067
date_rec_from=11%2F01%2F2021&date_rec_to=11%2F30%2F2021 27432
date_rec_from=10%2F01%2F2021&date_rec_to=10%2F31%2F2021 32867
date_rec_from=09%2F01%2F2021&date_rec_to=09%2F30%2F2021 37895
date_rec_from=08%2F01%2F2021&date_rec_to=08%2F31%2F2021 43295
date_rec_from=07%2F01%2F2021&date_rec_to=07%2F31%2F2021 49217
date_rec_from=06%2F01%2F2021&date_rec_to=06%2F30%2F2021 55193
date_rec_from=05%2F01%2F2021&date_rec_to=05%2F31%2F2021 60692
date_rec_from=04%2F01%2F2021&date_rec_to=04%2F30%2F2021 66505
date_rec_from=03%2F01%2F2021&date_rec_to=03%2F31%2F2021 73205
date_rec_from=02%2F01%2F2021&date_rec_to=02%2F28%2F2021 78458
date_rec_from=01%2F01%2F2021&date_rec_to=01%2F31%2F2021 84068
date_rec_from=12%2F01%2F2020&date_rec_to=12%2F31%2F2020 89599
date_rec_

In [5]:
#records as of 4:10 pm on March 25=238965
df.to_csv(f"data/foil_records_20180101_{str(date())}.csv", index=False)

In [6]:
monthly_records=monthly_count.copy()

monthly_records['month_count']=monthly_records.foil_count.diff()
monthly_records.month_count.iloc[0]=monthly_records.foil_count.iloc[0]

monthly_records['start_date']=pd.to_datetime(monthly_records.replace('date_rec_from=', "", regex=True)\
.replace("date_rec_to", "", regex=True).replace("%2F", "-",regex=True).date.str.split("&=", expand=True)[0])
monthly_records['end_date']=pd.to_datetime(monthly_records.replace('date_rec_from=', "", regex=True)\
.replace("date_rec_to", "", regex=True).replace("%2F", "-",regex=True).date.str.split("&=", expand=True)[1])

monthly_records=monthly_records.sort_values("start_date", ascending=False)[[
    'start_date', "end_date", 'month_count']].reset_index(drop=True)

monthly_records.to_csv(f"data/monthly_records_20180101_{str(date())}.csv", index=False)

/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
